In [1]:
import "fmt"

### Declaration Styles

#### Aggregate similar types
Following declarations are equivalent

In [2]:
func f(a int, b int, c int, s1 string,s2 string){
    
}

In [3]:
func f (a,b,c int, s1,s2 string){
    
}

### Unused parameters

In [4]:
func f(used int,_ int){ // second parameter unused, maybe we want to use the pointer to function generically, 
    //but for our operation only single parameter is sufficient
    
}

In [5]:
func f(int,string) float64{ // both the paramteters are unusewd
    return 3.1415
}

In [6]:

fmt.Println(same(4))

ERROR: repl.go:2:13: undefined identifier: same

### Multiple returns

In [7]:
func digits(twoDigitNumber int) (int,int){
    return twoDigitNumber/10,twoDigitNumber%10
}

fmt.Println(digits(34))

first,second := digits(56)

fmt.Printf("sum of digits = %d\n",first+second)

3 4
sum of digits = 11


19 <nil>

#### Cascaded returns

In [8]:
func digitsInSum(number int) (int,int){
    first,second := digits(number)
    return digits(first+second) // collects multiple return values and returns them as multiple values in the same statement
}

fmt.Println(digitsInSum(99))

1 8


4 <nil>

#### Named return values

In [9]:
func sumProduct(a,b int) (sum int, product int){
    return a+b,a*b
}

fmt.Println(sumProduct(3,4))

7 12


5 <nil>

#### Bare returns

In [10]:
func div(a,b int) (div float32,undefined bool){
    
    // default valuse for div = 0 and undefined = false
    if(b==0){
        undefined = true
        return // return whatever current values are there for return types
    }
    div = float32(a)/float32(b)
    return // return whatever current values are there for return types
}

fmt.Println(div(22,7))
fmt.Println(div(3,0))
fmt.Println(div(0,0))

3.142857 false
0 true
0 true


7 <nil>

### Errors

Error handling strategies:

1. Add more context to the error returned from called function which is relatable to the input to the current function , and finally return it to the calling function.
2. retry the call to the function returning error
3. Halt the program completely, in case of fatal problems
4. Just log and move on for minor problems
5. Ignore the errors completely for errors in non significant actions

Flow of GO functions, call the function, first check errors, process them and return in case of failures. Finally handle the successful condition.


### Function values

In [11]:
func add(a,b int) int{
    return a+b
}

func sub(a,b int) int{
    return a-b
}

func mul(a,b int) int{
    return a*b
}

func neg(a int) int{
    return -a
}

a,b := 3,4

fun := add // assign a func to a variable

fmt.Println(fun(a,b))

fun = sub // reassign

fmt.Println(fun(a,b))

fun = mul

fmt.Println(fun(a,b))

// fun = neg // incompatible types in assignment: func(int, int) int = func(int) int

7
-1
12


3 <nil>

#### Declaration of function flag

In [12]:
var f func(int) int

// f(3) // nil pointer dereference, as the variable is not initialized

#### Parameterized bahaviors

In [13]:
func operate(a,b int, operation func(int,int) int ) int{
    return operation(a,b)
}

fmt.Println(operate(4,5,add))
fmt.Println(operate(4,5,mul))
fmt.Println(operate(4,5,sub))

9
20
-1


3 <nil>

In [14]:
import "strings"

func add1(a rune)rune{
    return a+1
}

fmt.Println(strings.Map(add1,"dmbqxosdc")) // Library function provided with func

encrypted


10 <nil>

#### Anonymous functions

In [15]:
fmt.Println(strings.Map(func(r rune) rune {return r + 1},"dmbqxosdc" ))

// Note that function doesn't have name

encrypted


10 <nil>

##### Nested anonymous functions

In [16]:
func prefixer(pre string) func(string) string{ // pre is captured in the function and used every time
    return func(str string) string{
        return pre+str
    }
}

mr := prefixer("Mr.")

fmt.Println(mr("Suhas"))

fmt.Println(mr("Ashok"))

ms := prefixer("Ms.")

fmt.Println(ms("Suhas"))

fmt.Println(ms("Komal"))

Mr.Suhas
Mr.Ashok
Ms.Suhas
Ms.Komal


9 <nil>

In [17]:
func callCounter(fun func(int) int) func(int)int{
    count := 1 // count is now embedded in the function
    return func(num int) int{
        fmt.Printf("invoke time: %d\n",count)
        count++
        return fun(num)
    }
      
}

func square(i int)int{
    return i*i
}

square := callCounter(square) // Decorating the original function with the wrapper

square(4)
square(5)

func double(i int)int{
    return i+i
}

double := callCounter(double) // will get a new count as the callCounter is invoked afresh

double(8) 

invoke time: 1
invoke time: 2
invoke time: 1


16

##### NOTE : because of the hidden variable references, functions are reference types and therefore cannot be compared.

In [18]:
func fun(i int) func()int{
    retfun := func()int{return i}
    i++  // Note : even if i is captured before increment in the retfun, it is captured by its address
    // Therefore this increment is reflected in the retfun call 
    return retfun
}

fmt.Println(fun(4)()) 

5


2 <nil>

In [19]:
func squareProvider() []func()int{
    squares := make([]func()int,0,10)
    for i:=0;i<10;i++{
        squares = append(squares,func()int{return i*i}) // i is captured by address, so changes on each iteration
        // and all collected functions get the updated value of i
    }
    return squares
}

for _,squarer := range squareProvider(){
    fmt.Println(squarer())  
}

100
100
100
100
100
100
100
100
100
100


In [20]:
func perfectSquareProvider() []func()int{
    squares := make([]func()int,0,10)
    for i:=0;i<10;i++{
        currentI := i  // This is IMPORTANT, being a scope level variable, it is created afresh for each iteration of the loop
        squares = append(squares,func()int{return currentI*currentI}) // i is captured by address, so changes on each iteration
        // and all collected functions get the updated value of i
    }
    return squares
}

for _,squarer := range perfectSquareProvider(){
    fmt.Println(squarer())  
}

0
1
4
9
16
25
36
49
64
81


### Variadic functions

In [21]:
func sum(vals ...int) int{
    total := 0
    for _,val := range vals{
        total+=val
    }
    return total
}

fmt.Println(sum())
fmt.Println(sum(1))

fmt.Println(sum(1,2,3))

slice := []int{4,2,7}

fmt.Println(sum(slice...)) // passing a slice for a variadic, similar as spread operator

0
1
6
13


3 <nil>

### Defer

Can be thought as java finally block with function scope

In [22]:
func withDefer(){
    fmt.Println("At start")
    defer fmt.Println("finished")
    fmt.Println("At end")
}

withDefer()

At start
At end
finished


In [23]:
func withMultiDefer(){
    fmt.Println("At start")
    // The order is reversed
    defer fmt.Println("finished 1")
    defer fmt.Println("finished 2")
    defer fmt.Println("finished 3")
    fmt.Println("At end")
}

withMultiDefer()

// A usecase could be : obtaining a statement from a connection, need to close the statement (which is obtained 
// after getting the connection) needs to be closed first, then the connection needs to be closed.

At start
At end
finished 3
finished 2
finished 1


##### Pairing on-entry and on-exit operations

In [24]:
import "time"

func longOperation(){
    defer timeIt()() // timeIt() is evaluated first, and timeIt()() is run as deferred 
    time.Sleep(3*time.Second)
}

func timeIt() func(){
    start:= time.Now()
    return func(){
        fmt.Printf("Execution time: %s\n",time.Since(start))
    }
}

longOperation()

Execution time: 3.001277728s


##### Trace input and outputs

In [25]:
func typeOf(x int) (detectedType string){
    
    // the named result argument is captured in the function
    defer func(){fmt.Printf(" %d is detected as %s \n",x,detectedType)}()
    
    if(x < 0){
        return "-"
    }else if(x >0){
        return "+"
    }
    return "0"
}

typeOf(5)

 5 is detected as + 


+

### Panic and Recover

In [26]:
func caller(){
    
    divideBy(0)
    fmt.Println("Normal flow after recovery")
    divideBy(5)
}

func divideBy(z int) int{
    
    defer func(){
        // This is kind of catch block enclosing this function
        if p:=recover();p!=nil{
            fmt.Println("recovered from panic :",p)
        }
    }()
    ret := 5/z
    fmt.Printf("division %d\n",ret)
    return ret
}

caller()

recovered from panic : runtime error: integer divide by zero
Normal flow after recovery
division 1
